# Notebook: Thu thập dữ liệu khách hàng và danh mục tỉnh/ TP từ Pancake POS API

## 1. Lấy thông tin khách hàng

### 1.1. Import thư viện & Config
### Giải thích
- Import các thư viện cần thiết (`os`, `json`, `requests`, `pandas`, `sqlalchemy`, …).  
- Đọc các biến môi trường từ `.env` (API key, thông tin DB, shop_id).  
- Giúp bảo mật thông tin thay vì ghi trực tiếp trong code.  

In [1]:
import os
import json
import time
import random
import requests
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import LONGTEXT, VARCHAR
from sqlalchemy.types import BigInteger, DateTime
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường
load_dotenv()

API_KEY   = os.getenv("API_KEY")
DB_USER   = os.getenv("DB_USER")
DB_PASS   = os.getenv("DB_PASS")
DB_HOST   = os.getenv("DB_HOST")
DB_PORT   = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")
SHOP_ID   = os.getenv("SHOP_ID")  # SHOP_ID trong .env


### 1.2. Kết nối tới database Bronze
### Giải thích
- Sử dụng `SQLAlchemy` để tạo engine kết nối MySQL.  
- Database đích: schema Bronze, nơi lưu dữ liệu thô (raw).  
- In ra thông báo khi kết nối thành công.  

In [18]:
engine_bronze = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}"
)

print(f"✅ Kết nối thành công tới Bronze: {DB_BRONZE}")


✅ Kết nối thành công tới Bronze: winner_bronze


### 1.3. Hàm lấy một trang dữ liệu khách hàng từ API
### Giải thích
- Hàm `get_customers_page`:
  - Gọi API `/customers` với `shop_id`, `page`, `page_size`.
  - Trả về: danh sách khách hàng (`data`) và tổng số trang (`total_pages`).  
  - Có cơ chế retry tối đa 3 lần nếu lỗi (timeout hoặc HTTP error).  

In [3]:
def get_customers_page(shop_id, page=1, page_size=1000, max_retries=3):
    """
    Gọi API /shops/{shop_id}/customers để lấy một trang khách hàng
    """
    url = f"https://pos.pages.fm/api/v1/shops/{shop_id}/customers"
    params = {"api_key": API_KEY, "page": page, "page_size": page_size}
    
    for attempt in range(1, max_retries + 1):
        try:
            resp = requests.get(url, params=params, timeout=60)
            if resp.status_code == 200:
                j = resp.json()
                data = j.get("data", [])
                total_pages = j.get("total_pages") or j.get("meta", {}).get("total_pages")
                return data, total_pages
            else:
                print(f"⚠️ Lỗi {resp.status_code} tại page {page}, attempt {attempt}")
        except Exception as e:
            print(f"⚠️ Exception tại page {page}: {e}, attempt {attempt}")
        time.sleep(5 * attempt)
    return [], None


### 1.4. Hàm ghi dữ liệu batch vào MySQL
### Giải thích
- Hàm `insert_customers_batch`:
  - Nhận DataFrame `df_batch`.
  - Ghi dữ liệu vào bảng `customers_raw` trong schema Bronze.
  - Các cột gồm: `shop_id`, `customer_id`, `raw_json`, `extracted_at`.  

In [4]:
def insert_customers_batch(df_batch):
    df_batch.to_sql(
        "customers_raw",
        con=engine_bronze,
        if_exists="append",
        index=False,
        dtype={
            "shop_id": BigInteger(),
            "customer_id": VARCHAR(50),   # id khách có thể là chuỗi
            "raw_json": LONGTEXT(),
            "extracted_at": DateTime()
        }
    )


### 1.5. Hàm fetch & load nhiều trang khách hàng
### Giải thích
- Hàm `fetch_and_load_customers`:
  - Chạy vòng lặp qua từng trang API.
  - Chuyển đổi dữ liệu về DataFrame.
  - Lưu batch vào MySQL bằng `insert_customers_batch`.
  - Có delay ngẫu nhiên (1-3s) để tránh bị rate-limit.  

In [5]:
def fetch_and_load_customers(shop_id, start_page=1, page_size=1000):
    page = start_page
    while True:
        customers, total_pages = get_customers_page(shop_id, page, page_size)
        if not customers:
            print("📦 Không còn dữ liệu tại page", page)
            break

        df_batch = pd.DataFrame([{
            "shop_id": shop_id,
            "customer_id": cust.get("id"),
            "raw_json": json.dumps(cust, ensure_ascii=False),
            "extracted_at": datetime.now()
        } for cust in customers])

        insert_customers_batch(df_batch)
        print(f"✅ Page {page}/{total_pages} - Loaded {len(customers)} customers")

        if total_pages and page >= total_pages:
            break

        time.sleep(random.uniform(1, 3))  # ngủ ngẫu nhiên để tránh bị block
        page += 1


### 1.6. Chạy tiến trình thu thập dữ liệu
### Giải thích
- Gọi hàm `fetch_and_load_customers` để thu thập toàn bộ dữ liệu khách hàng.
- Kết quả: bảng `customers_raw` trong schema Bronze chứa dữ liệu thô từ API.  


In [6]:
fetch_and_load_customers(SHOP_ID, start_page=1, page_size=1000)


✅ Page 1/37 - Loaded 1000 customers
✅ Page 2/37 - Loaded 1000 customers
✅ Page 3/37 - Loaded 1000 customers
✅ Page 4/37 - Loaded 1000 customers
✅ Page 5/37 - Loaded 1000 customers
✅ Page 6/37 - Loaded 1000 customers
✅ Page 7/37 - Loaded 1000 customers
✅ Page 8/37 - Loaded 1000 customers
✅ Page 9/37 - Loaded 1000 customers
✅ Page 10/37 - Loaded 1000 customers
✅ Page 11/37 - Loaded 1000 customers
✅ Page 12/37 - Loaded 1000 customers
✅ Page 13/37 - Loaded 1000 customers
✅ Page 14/37 - Loaded 1000 customers
✅ Page 15/37 - Loaded 1000 customers
✅ Page 16/37 - Loaded 1000 customers
✅ Page 17/37 - Loaded 1000 customers
✅ Page 18/37 - Loaded 1000 customers
✅ Page 19/37 - Loaded 1000 customers
✅ Page 20/37 - Loaded 1000 customers
✅ Page 21/37 - Loaded 1000 customers
✅ Page 22/37 - Loaded 1000 customers
✅ Page 23/37 - Loaded 1000 customers
✅ Page 24/37 - Loaded 1000 customers
✅ Page 25/37 - Loaded 1000 customers
✅ Page 26/37 - Loaded 1000 customers
✅ Page 27/37 - Loaded 1000 customers
✅ Page 28/

--- 
## 2. Lấy danh mục tỉnh/thành phố từ Pancake POS API
Trong phần này, ta sẽ gọi endpoint `/geo/provinces` để lấy danh mục tỉnh/thành phố ở Việt Nam (country_code = "84").  
Kết quả sẽ được lưu vào bảng `province_raw` trong **tầng Bronze**.

### 2.1. Khai báo BASE_URL và headers (Authorization)
### Giải thích
- `BASE_URL`: đường dẫn gốc của Pancake POS API.  
- `headers`: chứa Authorization token (`Bearer API_KEY`) để xác thực khi gọi API.  


In [14]:
BASE_URL = "https://pos.pages.fm/api/v1"
headers = {"Authorization": f"Bearer {API_KEY}"}

### 2.2. Hàm gọi API chung
### Giải thích
- Hàm `fetch_api` giúp tái sử dụng cho nhiều endpoint.  
- Nhận vào `endpoint` và `params` → gọi API → trả về phần `"data"`.  
- Có xử lý lỗi (`try/except`) và in ra thông báo nếu thất bại.  


In [15]:
# Hàm gọi API

def fetch_api(endpoint, params=None):
    url = f"{BASE_URL}{endpoint}"
    try:
        res = requests.get(url, headers=headers, params=params, timeout=30)
        res.raise_for_status()
        return res.json().get("data", [])
    except Exception as e:
        print(f"❌ Lỗi khi gọi API {url}: {e}")
        return []


### 2.3. Hàm lấy danh mục tỉnh/thành phố
### Giải thích
- Hàm `get_provinces` gọi API `/geo/provinces`.  
- Dữ liệu trả về được chuyển thành `DataFrame` với 2 cột:
  - `province_id`
  - `province_name`  
- In ra số lượng tỉnh thành lấy được.  


In [16]:
# Lấy danh mục tỉnh/thành phố

def get_provinces():
    provinces = fetch_api("/geo/provinces", params={"country_code": "84"})
    print(f"✅ Lấy được {len(provinces)} tỉnh/thành")

    df = pd.DataFrame([{
        "province_id": p["id"],
        "province_name": p["name"]
    } for p in provinces])

    return df

# Chạy thử
df_provinces = get_provinces()
df_provinces.head()


✅ Lấy được 63 tỉnh/thành


,province_id,province_name
0,805,An Giang
1,221,Bắc Giang
2,207,Bắc Kạn
3,821,Bạc Liêu
4,106,Bắc Ninh


### 2.4. Ghi dữ liệu provinces vào MySQL (Bronze)
### Giải thích
- Sử dụng `to_sql` để ghi dữ liệu vào bảng `province_raw`.  
- `if_exists="replace"`: xóa bảng cũ và ghi lại (nếu muốn cộng dồn thì đổi sang `"append"`).  
- Kiểu dữ liệu trong MySQL:
  - `province_id`: VARCHAR(20).  
  - `province_name`: VARCHAR(255).  


In [19]:
df_provinces.to_sql(
    name="province_raw",   # tên bảng trong MySQL
    con=engine_bronze,            # kết nối tới Bronze DB
    if_exists="replace",          # ghi đè, dùng "append" nếu muốn thêm dữ liệu
    index=False,
    dtype={
        "province_id": VARCHAR(20),
        "province_name": VARCHAR(255)
    }
)
print("✅ Đã lưu bảng province_raw vào Bronze DB")


✅ Đã lưu bảng province_raw vào Bronze DB
